In [1]:
import yaml
import re

In [2]:
ymlDoc = yaml.load(open('modules.yaml'))

In [3]:
print (ymlDoc)

{'module': {'ABC_TOP': {'inputs': [['bit<0-0>', 'n_clock', 'Global Clock'], ['bit<0-10>', 'v_clock', 'Vlobal Clock']], 'outputs': [['bit<0-0>', 'n_clock', 'Global Clock'], ['bit<0-10>', 'v_clock', 'Vlobal Clock']]}}, 'Mapping': ['ABC.xyz > PQR.pqr', 'ABC.zyx > PQR.rqp']}


In [11]:
def genIOTxt (listPorts, listType, indent = 0):
    ''' Takes the list of Input or output ports and generates the RTL code for paramters of module '''
    assert listType == 'input' or listType == 'output', 'Unknown port list type: ' + listType
    
    txtPortsVerilog = '// ' + listType + '\n'
    for port in listPorts:
        bitWidth, name, comment = port
        lowBit, highbit = re.findall(r'\d+',bitWidth)
        txtPortsVerilog += ' ' * 4 * indent
        txtPortsVerilog += listType + ' [' + highbit + ':' + lowBit + '] ' + name + ' '
        if not listPorts.index(port) == len(listPorts) - 1: # Leave last comma
            txtPortsVerilog += ', '
        txtPortsVerilog += '// ' + comment + '\n'
        
    return txtPortsVerilog



In [12]:
print (genIOTxt(ymlDoc['module']['ABC_TOP']['inputs'], 'input', 1))

// input
    input [0:0] n_clock , // Global Clock
    input [10:0] v_clock // Vlobal Clock



In [6]:
def genModuleTxt(moduleName, moduleDetails):
    ''' Takes the module details and returns the RTL code for module definition'''
    txtModule = 'module ' + moduleName + '(\n'
    txtModule += genIOTxt(ymlDoc['module']['ABC_TOP']['inputs'], 'input' ,1 )
    txtModule += '\n'
    txtModule += genIOTxt(ymlDoc['module']['ABC_TOP']['outputs'], 'output', 1)
    txtModule += 'endmodule' + '\n'
    return txtModule

In [7]:
print (genModuleTxt('ABC_TOP',ymlDoc['module']['ABC_TOP']))

module ABC_TOP(
// input
 input [0:0] n_clock , // Global Clock
 input [10:0] v_clock // Vlobal Clock

// output
 output [0:0] n_clock , // Global Clock
 output [10:0] v_clock // Vlobal Clock
endmodule



In [22]:
portMappingInp = {'xyz':'pqr','zyx':'rqp' }
portMappingOut = {'xyzo':'pqro','zyxo':'rqpo' }

In [27]:
def genConnTxt(mapInPorts,mapOutPorts, indentLevel = 0):
    indent = ' ' * indentLevel * 4 
    connTxt = ''
    # Inputs
    connTxt += indent + '// Inputs \n'
    for i,inConn in enumerate(mapInPorts):
        connTxt += indent + '.' + inConn + ' (.' + mapInPorts[inConn] + ')'
        if not (i == len(mapInPorts) -1 and len(mapOutPorts) == 0): # last connection drop comma if no output conns
            connTxt += ','
        connTxt += '\n'

    # Outputs
    connTxt += '\n' + indent + '// Outputs \n'            
    for i,outConn in enumerate(mapOutPorts):
        connTxt += indent + '.' + outConn + ' (.' + mapOutPorts[outConn] + ')'
        if not (i == len(mapInPorts) - 1): # last connection drop comma if no output conns
            connTxt += ','
        connTxt += '\n'        
    
    return connTxt

In [28]:
print (genConnTxt(portMappingInp, portMappingOut , 1))

    // Inputs 
    .xyz (.pqr),
    .zyx (.rqp),

    // Outputs 
    .xyzo (.pqro),
    .zyxo (.rqpo)

